In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import tensorflow_datasets as tfds
import numpy as np
from sklearn.metrics import confusion_matrix


In [2]:
# ✅ Load EuroSAT dataset
dataset, info = tfds.load("eurosat/rgb", as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/incomplete.1JHKZF_2.0.0/eurosat-train.tfrecord*...:   0%|     …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.


In [3]:
# ✅ Image Transformations for ConvNeXt
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# ✅ Convert TF dataset to PyTorch Dataset
class EuroSATDataset(torch.utils.data.Dataset):
    def __init__(self, tf_dataset):
        self.data = list(tf_dataset.as_numpy_iterator())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        image = transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# ✅ Split dataset
train_dataset = EuroSATDataset(dataset["train"])
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ✅ Load ConvNeXt Model (Feature Extractor)
device = "cuda" if torch.cuda.is_available() else "cpu"
convnext_model = models.convnext_large(pretrained=True)
convnext_model.classifier = nn.Identity()  # Remove final classification layer
convnext_model = convnext_model.to(device)

# ✅ Define Liquid Neural Network Classifier
class LiquidNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LiquidNN, self).__init__()
        self.liquid_layer = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.liquid_layer(x)

lnn_model = LiquidNN(1536, info.features["label"].num_classes).to(device)  # ConvNeXt output dim = 1536

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Large_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_large-ea097f82.pth" to /root/.cache/torch/hub/checkpoints/convnext_large-ea097f82.pth
100%|██████████| 755M/755M [00:08<00:00, 90.3MB/s]


In [5]:
import time

# ✅ Training Function with Timing
def train_model(feature_extractor, lnn_model, train_loader, epochs=5):
    feature_extractor.eval()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(lnn_model.parameters(), lr=0.0005, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    for epoch in range(epochs):
        start_time = time.time()  # Start timing
        lnn_model.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            with torch.no_grad():
                features = feature_extractor(images)
                features = features.view(features.size(0), -1)

            optimizer.zero_grad()
            outputs = lnn_model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        scheduler.step()
        elapsed_time = time.time() - start_time  # Compute elapsed time
        print(f"Epoch {epoch+1}: Loss: {total_loss / len(train_loader):.4f}, "
              f"Accuracy: {100 * correct / total:.2f}%, Time: {elapsed_time:.2f} sec")


In [6]:

# ✅ Train the Model
train_model(convnext_model, lnn_model, train_loader, epochs=50)

Epoch 1: Loss: 0.3656, Accuracy: 89.52%, Time: 638.71 sec
Epoch 2: Loss: 0.1897, Accuracy: 94.17%, Time: 633.14 sec
Epoch 3: Loss: 0.1618, Accuracy: 94.70%, Time: 624.84 sec
Epoch 4: Loss: 0.1428, Accuracy: 95.22%, Time: 625.27 sec
Epoch 5: Loss: 0.1331, Accuracy: 95.36%, Time: 632.13 sec
Epoch 6: Loss: 0.1265, Accuracy: 95.64%, Time: 628.01 sec
Epoch 7: Loss: 0.1170, Accuracy: 96.07%, Time: 626.96 sec
Epoch 8: Loss: 0.1120, Accuracy: 96.15%, Time: 626.65 sec
Epoch 9: Loss: 0.1106, Accuracy: 96.37%, Time: 626.90 sec
Epoch 10: Loss: 0.1056, Accuracy: 96.29%, Time: 630.78 sec
Epoch 11: Loss: 0.1006, Accuracy: 96.63%, Time: 625.93 sec
Epoch 12: Loss: 0.0934, Accuracy: 96.82%, Time: 627.01 sec
Epoch 13: Loss: 0.0970, Accuracy: 96.66%, Time: 634.54 sec
Epoch 14: Loss: 0.0919, Accuracy: 97.00%, Time: 627.88 sec
Epoch 15: Loss: 0.0898, Accuracy: 96.81%, Time: 625.80 sec
Epoch 16: Loss: 0.0852, Accuracy: 97.11%, Time: 625.87 sec
Epoch 17: Loss: 0.0822, Accuracy: 97.20%, Time: 632.91 sec
Epoch 

In [7]:
# ✅ Testing Function
def test_model(feature_extractor, model, test_loader):
    feature_extractor.eval()
    model.eval()
    criterion = nn.CrossEntropyLoss()

    correct, total, total_loss = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            features = features.view(features.size(0), -1)

            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    avg_loss = total_loss / len(test_loader)
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy, avg_loss, np.array(all_labels), np.array(all_preds)

In [8]:

# ✅ Evaluate the Model
test_accuracy, test_loss, y_true, y_pred = test_model(convnext_model, lnn_model, test_loader)

Test Loss: 0.0815, Test Accuracy: 97.26%


In [9]:
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[570   0   2   2   0   3   7   0   5   3]
 [  0 612   2   0   0   1   0   0   0   2]
 [  1   2 573   0   0   3   9   0   0   1]
 [  2   1   1 477   2   0   1   1  12   0]
 [  0   0   0   0 474   0   0   4   0   0]
 [  3   2   6   2   0 363   0   0   1   0]
 [  6   0  19   3   1   2 490   1   1   0]
 [  0   0   2   0   3   0   0 588   0   0]
 [  1   0   2  11   2   1   1   0 479   1]
 [  0   3   1   0   0   1   0   0   5 626]]


In [10]:
import torch
import os
from google.colab import files  # If using Google Colab

def save_and_download_model(model, filename="trained_model.pth"):
    """
    Saves the trained model and provides a download link.

    Args:
        model (torch.nn.Module): The trained PyTorch model.
        filename (str): The filename to save the model.
    """
    # Save the model
    torch.save(model.state_dict(), filename)
    print(f"Model saved as {filename}")

    # Check if running in Google Colab
    if "google.colab" in str(get_ipython()):
        files.download(filename)  # Download the file automatically in Colab
    else:
        print(f"Download manually from the current directory: {os.getcwd()}/{filename}")

# Example usage:
save_and_download_model(lnn_model, "liquid_neural_network.pth")

Model saved as liquid_neural_network.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>